In [2]:
# import os
# import openai
# from dotenv import load_dotenv, find_dotenv
# # read local .env file
# _ = load_dotenv(find_dotenv())
# openai.api_key = os.environ['OPENAI_API_KEY']
# # print(openai.api_key)

import pydantic

print(f'{pydantic.__version__}')

2.6.0


In [19]:
llm_model = "gpt-3.5-turbo"


# It looks like this behavior is related to recent Pydantic version.
# To make it work, I had to downgrade Pydantic to 1.10.9 which downgrade in turn langchain to 0.0.347

In [5]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [6]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [7]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

/home/ypearson/.pyenv/versions/3.12.0/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/home/ypearson/.pyenv/versions/3.12.0/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [8]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

/home/ypearson/.pyenv/versions/3.12.0/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [8]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

In [9]:
from langchain.evaluation.qa import QAGenerateChain

In [12]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=llm_model))

In [13]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/home/ypearson/.pyenv/versions/3.12.0/lib/python3.12/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [14]:
new_examples[0]

{'qa_pairs': {'query': "What is the weight of each pair of Women's Campside Oxfords?",
  'answer': "The approximate weight of each pair of Women's Campside Oxfords is 1 lb. 1 oz."}}

In [15]:
examples += new_examples

In [18]:
qa.run(examples[0]["query"])



> Entering new RetrievalQA chain...


ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [-0.0064034... -0.028614587546074315]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [-0.0064034... -0.028614587546074315]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing